# Libraries

In [17]:
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path

from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings

from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chat_models import init_chat_model
from typing import List, Optional
from pydantic import BaseModel, Field
from tqdm import tqdm

# Load .env


In [18]:
from dotenv import load_dotenv

# Setting Langsmith tracing

In [19]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "Rag_travel_planner_v1.0"

# Load data Dfs

In [20]:
landmark_prices = pd.read_csv('data/egypt_v0.1.csv')
places_api_data = pd.read_csv('data/places_details.csv')


# Store Dfs in langchain Documents

In [21]:
documents = []

for _, row in landmark_prices.iterrows():
    text = f"""
    Governorate: {row['Governorate/City']}
    Site: {row['Place']}
    Egyptian Ticket: {row['Egyptian']}
    Egyptian Student Ticket: {row['EgyptianStudent']}
    Foreign Ticket: {row['Foreign']}
    Foreign Student Ticket: {row['ForeignStudent']}
    Visiting Times: {row['VisitingTimes']}
    """
    documents.append(Document(page_content=text, metadata={"source": 'landmark_prices'}))

In [22]:
for _, row in places_api_data.iterrows():
    text = f"""
    Place Name: {row['displayName.text']}
    Place Primary Type: {row['primaryTypeDisplayName.text']}
    Place Types: {row['types']}
    Place Price: {row['priceRange.endPrice.units']}
    Place Price Level: {row['priceLevel']}
    Place Location: {row['formattedAddress']}
    Place Star Rating: {row['rating']}
    Place website: {row['websiteUri']}
    """
    documents.append(Document(page_content=text, metadata={"source": 'Places_api', 'Type': f'{row['primaryTypeDisplayName.text']}', 'city': f'{row['formattedAddress']}'}))
    

# Embedd Documents (text ---> vectors of numbers)

In [23]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


In [24]:
path = Path('faiss_mpnetv2_v1.0')

if not path.exists():
    vectorstore = FAISS.from_documents(documents, embeddings)
    vectorstore.save_local('faiss_mpnetv2_v1.0')



vectorstore.load_local('faiss_mpnetv2_v1.0', embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(search_kwargs={"k": 50})

# Instantiating the LLM model with Groq provider.

In [37]:
llm_model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq", temperature=0)
# llm_model = init_chat_model('deepseek-r1-distill-llama-70b', model_provider='groq', temperature=0)

# Prompt Template

In [26]:
prompt_template = PromptTemplate(
    input_variables=["context", "user_query", "favorite_places", "visitor_type", "num_days", "budget"],
    template="""You are a helpful travel planner AI.
Use the context below, which contains information about ticket prices, place descriptions, restaurant details, and art gallery information.

Context:
{context}

User Query:
{user_query}

Additional Preferences:
- Favorite types of places: {favorite_places}
- Visitor type: {visitor_type} (e.g., Egyptian, Egyptian student, Foreign, or foreign student)
- Number of travel days: {num_days}
- Overall budget for all days: {budget} EGP
- Exclude hotels from the plan.
- Ensure that the itinerary includes at least 3 meals per day.

Based on the above, return a detailed {num_days}-day travel itinerary with approximate costs and suggestions. If some details are missing, make reasonable assumptions and indicate them.
"""
)


# Structured Output

In [34]:
json_schema = {
    "title": "TravelItinerary",
    "description": "A structured travel itinerary for the user.",
    "type": "object",
    "properties": {
        "days": {
            "type": "array",
            "description": "List of days with planned activities.",
            "items": {
                "type": "object",
                "properties": {
                    "day": {"type": "string", "description": "Theme of the Day or Day label, e.g., 'Day 1'"},
                    "activities": {
                        "type": "array",
                        "description": "Activities planned for the day.",
                        "items": {
                            "type": "object",
                            "properties": {
                                "time": {"type": "string", "description": "Time of the activity"},
                                "activity": {"type": "string", "description": "Name of the activity"},
                                "location": {"type": "string", "description": "Location name"},
                                "price_range": {"type": "string", "description": "Price range or cost"},
                            },
                            "required": ["time", "activity", "location"]
                        }
                    },
                    "approximate_cost": {"type": "string", "description": "Total cost for the day"}
                },
                "required": ["day", "activities", "approximate_cost"]
            }
        },
        "total_approximate_cost": {
            "type": "string",
            "description": "Total cost for the trip"
        },
        "notes": {
            "type": "string",
            "description": "Any additional notes or assumptions"
        }
    },
    "required": ["days", "total_approximate_cost"]
}

In [28]:
class Activity(BaseModel):
    time: str = Field(..., description="Time of the activity")
    activity: str = Field(..., description="Name of the activity")
    location: str = Field(..., description="Location name")
    price_range: Optional[str] = Field(None, description="Price range or cost")

class DayPlan(BaseModel):
    day: str = Field(..., description="Theme of the Day or Day label, e.g., 'Day 1'")
    activities: List[Activity] = Field(..., description="Activities planned for the day")
    approximate_cost: str = Field(..., description="Total cost for the day")

class TravelItinerary(BaseModel):
    days: List[DayPlan] = Field(..., description="List of days with planned activities")
    total_approximate_cost: str = Field(..., description="Total cost for the trip")
    notes: Optional[str] = Field(None, description="Any additional notes or assumptions")

In [38]:
structured_llm = llm_model.with_structured_output(json_schema, include_raw=True)


# Generate a structured travel plan 

In [30]:

def generate_travel_plan(user_query, favorite_places, visitor_type, num_days, budget):
    # 1. Retrieve relevant docs
    docs = retriever.get_relevant_documents(user_query)
    context_text = "\n".join([doc.page_content for doc in docs])
    # 2. Build the final prompt with the additional variables
    prompt = prompt_template.format(
        context=context_text,
        user_query=user_query,
        favorite_places=favorite_places,
        visitor_type=visitor_type,
        num_days=num_days,
        budget=budget
    )
    # 3. Call the LLM
    response = structured_llm.invoke(prompt)
    return response


In [31]:
user_query = "Plan a 3-day trip in Luxor with visits to cultural sites, art galleries, and dining(restaurents) options."
favorite_places = "Cultural sites, historical landmarks, art galleries"
visitor_type = "Foreign"  # or "Egyptian", "Egyptian student", "foreign student"
num_days = "3"
budget = "5000"  # Overall budget in EGP


In [39]:
travel_plan = generate_travel_plan(user_query, favorite_places, visitor_type, num_days, budget)
print(travel_plan)


{'raw': AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_y4hr', 'function': {'arguments': '{"days": [{"day": "Day 1", "activities": [{"activity": "Visit Luxor Museum", "location": "Kornish Al Nile, Luxor City, Luxor, Luxor Governorate 1362503, Egypt", "price_range": "400 EGP", "time": "9:00am - 1:00pm"}, {"activity": "Lunch at White Coffee & Restaraunt", "location": "Luxor, Karnak, Luxor, Luxor Governorate 1363024, Egypt", "price_range": "200 EGP", "time": "1:00pm - 2:30pm"}, {"activity": "Visit Luxor Art Gallery", "location": "Memnon Street, Al Bairat, Al Qarna, Luxor Governorate 1341472, Egypt", "price_range": "Free", "time": "3:00pm - 5:00pm"}, {"activity": "Dinner at Nubian House", "location": "Al Bairat, Luxor, Luxor Governorate 1345173, Egypt", "price_range": "300 EGP", "time": "7:00pm - 9:00pm"}], "approximate_cost": "1900 EGP"}, {"day": "Day 2", "activities": [{"activity": "Visit Karnak Temples", "location": "Karnak, Luxor, Luxor Governorate, Egypt", "price_